# Set initialization file
https://cccma.gitlab.io/classic/makeInputFiles.html

In [1]:
# Env: phd_v3
# To reload external files automatically (ex: utils)
%load_ext autoreload
%autoreload 2

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import proplot as plot # New plot library (https://proplot.readthedocs.io/en/latest/)
plot.rc['savefig.dpi'] = 300 # 1200 is too big! #https://proplot.readthedocs.io/en/latest/basics.html#Creating-figures
from scipy import stats
import xesmf as xe # For regridding (https://xesmf.readthedocs.io/en/latest/)
import calendar
import os

In [2]:
path_in = '/home/lalandmi/eccc/classic-develop/inputFiles/SnowMIP/FR-Cdp_test' # example file
path_out = '/home/lalandmi/eccc/classic-develop/inputFiles/SnowMIP/FR-Cdp_test'

In [3]:
ds = xr.open_dataset(path_in+'/FR-Cdp_init_run.nc')
ds

<xarray.Dataset>
Dimensions:            (ic: 5, icc: 12, iccp1: 13, iccp2: 14, icp1: 6, lat: 1, layer: 20, lon: 1, slope: 8, tile: 1)
Coordinates:
  * lat                (lat) float64 45.3
  * lon                (lon) float64 5.77
  * tile               (tile) int32 1
  * icp1               (icp1) int32 1 2 3 4 5 6
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * slope              (slope) int32 1 2 3 4 5 6 7 8
Data variables:
    ALBS               (tile, lat, lon) float64 ...
    ALIC               (tile, icp1, lat, lon) float64 ...
    ALVC               (tile, icp1, lat, lon) float64 ...
    CLAY               (tile, layer, lat, lon) float64 ...
    CMAS               (tile, ic, lat, lon) float64 ...
    Cmossmas           (tile, lat, lon) float64 ...
    DELZ               (layer) float64 ...
    DRN                (tile, lat, lon) float64 ...
    FARE               (tile, lat, lon) float64 ...
    FCAN               (tile, icp1, lat, lon) float64 ...
    GRO                (tile, lat, lon) float64 ...
    LNZ0               (tile, icp1, lat, lon) float64 ...
    ORGM               (tile, layer, lat, lon) float64 ...
    PAMN               (tile, ic, lat, lon) float64 ...
    PAMX               (tile, ic, lat, lon) float64 ...
    RCAN               (tile, lat, lon) float64 ...
    RHOS               (tile, lat, lon) float64 ...
    ROOT               (tile, ic, lat, lon) float64 ...
    SAND               (tile, layer, lat, lon) float64 ...
    SCAN               (tile, lat, lon) float64 ...
    SDEP               (tile, lat, lon) float64 ...
    SNO                (tile, lat, lon) float64 ...
    SOCI               (tile, lat, lon) float64 ...
    TBAR               (tile, layer, lat, lon) float64 ...
    TCAN               (tile, lat, lon) float64 ...
    THIC               (tile, layer, lat, lon) float64 ...
    THLQ               (tile, layer, lat, lon) float64 ...
    TPND               (tile, lat, lon) float64 ...
    TSNO               (tile, lat, lon) float64 ...
    ZPND               (tile, lat, lon) float64 ...
    bleafmas           (tile, icc, lat, lon) float64 ...
    dmoss              (tile, lat, lon) float64 ...
    fcancmx            (tile, icc, lat, lon) float64 ...
    grclarea           (lat, lon) float64 ...
    ipeatland          (tile, lat, lon) float64 ...
    lfstatus           (tile, icc, lat, lon) float64 ...
    litrmass           (tile, layer, iccp2, lat, lon) float64 ...
    litrmsmoss         (tile, lat, lon) float64 ...
    nbleafmas          (tile, icc, lat, lon) float64 ...
    ngleafmas_ns       (tile, icc, lat, lon) float64 ...
    ngleafmas_s        (tile, icc, lat, lon) float64 ...
    nh4_mass           (tile, iccp1, lat, lon) float32 ...
    nlitrmass          (tile, iccp1, lat, lon) float64 ...
    soilnmas           (tile, iccp1, lat, lon) float64 ...
    nmtest             (lat, lon) float64 ...
    no3_mass           (tile, iccp1, lat, lon) float32 ...
    soilpH             (tile, lat, lon) float32 ...
    nrootmass_ns       (tile, icc, lat, lon) float64 ...
    nrootmass_s        (tile, icc, lat, lon) float64 ...
    nstemmass_ns       (tile, icc, lat, lon) float64 ...
    nstemmass_s        (tile, icc, lat, lon) float64 ...
    pandays            (tile, icc, lat, lon) float64 ...
    slopefrac          (tile, slope, lat, lon) float64 ...
    soilcmas           (tile, layer, iccp2, lat, lon) float64 ...
    maxAnnualActLyr    (tile, lat, lon) float64 ...
    grwtheff           (tile, icc, lat, lon) float64 ...
    gleafmas_ns        (tile, icc, lat, lon) float64 ...
    gleafmas_s         (tile, icc, lat, lon) float64 ...
    stemmass_ns        (tile, icc, lat, lon) float64 ...
    stemmass_s         

In [4]:
ds.soilcmas.sum()

<xarray.DataArray 'soilcmas' ()>
array(19.77891896)

In [5]:
ctempfts = 'NdlEvgTr', 'NdlDcdTr', 'BdlEvgTr', 'BdlDCoTr', 'BdlDDrTr', 'CropC3', 'CropC4', 'GrassC3', 'GrassC4', 'Sedge', 'BdlEvgSh', 'BdlDCoSh'
fcancmx = {'NdlEvgTr': 0., 'NdlDcdTr': 0., 'BdlEvgTr': 0., 'BdlDCoTr': 0., 'BdlDDrTr': 0., 
           'CropC3': 0., 'CropC4': 0., 'GrassC3': 0., 'GrassC4': 0., 'Sedge': 0., 
           'BdlEvgSh': 0., 'BdlDCoSh': 0.}

for i, pft in enumerate(ctempfts):
    ds.fcancmx[0, i, 0, 0] = fcancmx[pft]
    print(pft + ' -> ' + str(fcancmx[pft]*100) + ' %')
    
ds.fcancmx[0, :, 0, 0]

NdlEvgTr -> 0.0 %
NdlDcdTr -> 0.0 %
BdlEvgTr -> 0.0 %
BdlDCoTr -> 0.0 %
BdlDDrTr -> 0.0 %
CropC3 -> 0.0 %
CropC4 -> 0.0 %
GrassC3 -> 0.0 %
GrassC4 -> 0.0 %
Sedge -> 0.0 %
BdlEvgSh -> 0.0 %
BdlDCoSh -> 0.0 %


<xarray.DataArray 'fcancmx' (icc: 12)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * icc      (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    units:      -
    long_name:  PFT fractional coverage per grid cell

## Set data for Col de Porte 

**5.4 Col de Porte, France**

Grassy meadow bordered by coniferous forest. Soils are 30% clay, 60% sand and 10% silt.

**Table 5.4: Col de Porte site characteristics**

| Short name | cdp |
|:-----------|:----|
| Location | 45.30ºN, 5.77ºE |
| Elevation | 1325 m |
| Snow-free albedo | 0.2 |
| Simulation period | 1 October 1994 to 30 September 2014 |
| Temperature/humidity measurement height | 1.5 m above the snow surface |
| Wind measurement height | 10 m |
| Reference | Morin et al. (2012) |

In [6]:
site = 'FR-Cdp'

lat = 5.77
lon = 45.30

# These values can be attributed with the global grid if not available
SDEP = 12.0 # Soil permeable depth (m)
SOCI= 15.0 # Soil color index

# Those can be distributed along the soil layers if available
CLAY = 30. # Percentage clay content (%)
SAND = 60. # Percentage of sand content (%)
ORGM = 0. # Percentage organic matter content (%)
# If the sum does not reach 100 %, the left over will be attributed to silt

# ! List of CLASS-level PFTs
classpfts = 'NdlTr', 'BdlTr', 'Crops', 'Grass', 'BdlSh'
FCAN = {'NdlTr': 0. , 'BdlTr': 0., 'Crops': 0., 'Grass': 0., 'BdlSh': 0.} # max 1
# Maybe add a little fraction of trees (as it's bordered by coniferous -> TODO)

# ! List of CTEM PFTs
# ! **Note: 'BdlDCoTr' should be specified before 'BdlDDrTr' due to some code in competition.
ctempfts = 'NdlEvgTr', 'NdlDcdTr', 'BdlEvgTr', 'BdlDCoTr', 'BdlDDrTr', 'CropC3', 'CropC4', 'GrassC3', 'GrassC4', 'Sedge', 'BdlEvgSh', 'BdlDCoSh'
fcancmx = {'NdlEvgTr': 0., 'NdlDcdTr': 0., 'BdlEvgTr': 0., 'BdlDCoTr': 0., 'BdlDDrTr': 0., 
           'CropC3': 0., 'CropC4': 0., 'GrassC3': 0., 'GrassC4': 0., 'Sedge': 0., 
           'BdlEvgSh': 0., 'BdlDCoSh': 0.} # Max 1
# Check differences between C3 and C4 at Cdp? (TODO)

if sum(FCAN.values()) > 1: raise Exception("The sum of FCAN values needs to be lower than 1.")
if sum(fcancmx.values()) > 1: raise Exception("The sum of fcanmax values needs to be lower than 1.")

### Set lat/lon

In [7]:
with xr.set_options(keep_attrs=True):
    ds = ds.assign_coords(lon=(ds.lon*0+lat))
    ds = ds.assign_coords(lat=(ds.lat*0+lon))
ds

<xarray.Dataset>
Dimensions:            (ic: 5, icc: 12, iccp1: 13, iccp2: 14, icp1: 6, lat: 1, layer: 20, lon: 1, slope: 8, tile: 1)
Coordinates:
  * lat                (lat) float64 45.3
  * lon                (lon) float64 5.77
  * tile               (tile) int32 1
  * icp1               (icp1) int32 1 2 3 4 5 6
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * slope              (slope) int32 1 2 3 4 5 6 7 8
Data variables:
    ALBS               (tile, lat, lon) float64 0.84
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 30.0 30.0 ... 0.0 0.0
    CMAS               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    DELZ               (layer) float64 0.1 0.1 0.1 0.1 0.1 ... 5.0 15.0 30.0 5.0
    DRN                (tile, lat, lon) float64 0.1
    FARE               (tile, lat, lon) float64 1.0
    FCAN               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 1.0 0.0 0.0
    GRO                (tile, lat, lon) float64 0.0
    LNZ0               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0
    ORGM               (tile, layer, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    PAMN               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    PAMX               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    RCAN               (tile, lat, lon) float64 0.0
    RHOS               (tile, lat, lon) float64 283.3
    ROOT               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    SAND               (tile, layer, lat, lon) float64 60.0 60.0 ... -3.0 -3.0
    SCAN               (tile, lat, lon) float64 0.002509
    SDEP               (tile, lat, lon) float64 12.0
    SNO                (tile, lat, lon) float64 177.4
    SOCI               (tile, lat, lon) float64 15.0
    TBAR               (tile, layer, lat, lon) float64 -5.684e-14 ... 6.547
    TCAN               (tile, lat, lon) float64 -1.2
    THIC               (tile, layer, lat, lon) float64 0.007972 0.0 ... 0.0 0.0
    THLQ               (tile, layer, lat, lon) float64 0.286 0.2891 ... 0.0 0.0
    TPND               (tile, lat, lon) float64 0.0
    TSNO               (tile, lat, lon) float64 -1.31
    ZPND               (tile, lat, lon) float64 0.0
    bleafmas           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dmoss              (tile, lat, lon) float64 0.0
    fcancmx            (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    grclarea           (lat, lon) float64 100.0
    ipeatland          (tile, lat, lon) float64 0.0
    lfstatus           (tile, icc, lat, lon) float64 1.0 4.0 1.0 ... 0.0 0.0 0.0
    litrmass           (tile, layer, iccp2, lat, lon) float64 0.0 0.0 ... 0.0
    litrmsmoss         (tile, lat, lon) float64 0.0
    nbleafmas          (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ngleafmas_ns       (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ngleafmas_s        (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
    nlitrmass          (tile, iccp1, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    soilnmas           (tile, iccp1, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    nmtest             (lat, lon) float64 1.0
    no3_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
    soilpH             (tile, lat, lon) float32 5.835
    nrootmass_ns       (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nrootmass_s        (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0

### Set PFTs

In [8]:
ds.FCAN[0, :, 0, 0]

<xarray.DataArray 'FCAN' (icp1: 6)>
array([0., 0., 0., 1., 0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * icp1     (icp1) int32 1 2 3 4 5 6
Attributes:
    units:      -
    long_name:  Annual maximum fractional coverage of modelled area (read in ...

In [9]:
for i, pft in enumerate(classpfts):
    ds.FCAN[0, i, 0, 0] = FCAN[pft]
    print(pft + ' -> ' + str(FCAN[pft]*100) + ' %')
    
ds.FCAN[0, :, 0, 0]
# What is the 6th PFT? Bareground?

NdlTr -> 0.0 %
BdlTr -> 0.0 %
Crops -> 0.0 %
Grass -> 0.0 %
BdlSh -> 0.0 %


<xarray.DataArray 'FCAN' (icp1: 6)>
array([0., 0., 0., 0., 0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * icp1     (icp1) int32 1 2 3 4 5 6
Attributes:
    units:      -
    long_name:  Annual maximum fractional coverage of modelled area (read in ...

In [10]:
ds.fcancmx[0, :, 0, 0]

<xarray.DataArray 'fcancmx' (icc: 12)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * icc      (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    units:      -
    long_name:  PFT fractional coverage per grid cell

In [11]:
for i, pft in enumerate(ctempfts):
    ds.fcancmx[0, i, 0, 0] = fcancmx[pft]
    print(pft + ' -> ' + str(fcancmx[pft]*100) + ' %')
    
ds.fcancmx[0, :, 0, 0]
# What is the 6th PFT? Bareground?

NdlEvgTr -> 0.0 %
NdlDcdTr -> 0.0 %
BdlEvgTr -> 0.0 %
BdlDCoTr -> 0.0 %
BdlDDrTr -> 0.0 %
CropC3 -> 0.0 %
CropC4 -> 0.0 %
GrassC3 -> 0.0 %
GrassC4 -> 0.0 %
Sedge -> 0.0 %
BdlEvgSh -> 0.0 %
BdlDCoSh -> 0.0 %


<xarray.DataArray 'fcancmx' (icc: 12)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * icc      (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
Attributes:
    units:      -
    long_name:  PFT fractional coverage per grid cell

### Set soil color

In [12]:
ds.SOCI

<xarray.DataArray 'SOCI' (tile: 1, lat: 1, lon: 1)>
array([[[15.]]])
Coordinates:
  * lat      (lat) float64 45.3
  * lon      (lon) float64 5.77
  * tile     (tile) int32 1
Attributes:
    units:      index
    long_name:  Soil colour index

In [13]:
with xr.set_options(keep_attrs=True):
    ds['SOCI'][0, 0, 0] = SOCI
ds.SOCI

<xarray.DataArray 'SOCI' (tile: 1, lat: 1, lon: 1)>
array([[[15.]]])
Coordinates:
  * lat      (lat) float64 45.3
  * lon      (lon) float64 5.77
  * tile     (tile) int32 1
Attributes:
    units:      index
    long_name:  Soil colour index

### Set permeable depth

In [14]:
ds.SDEP

<xarray.DataArray 'SDEP' (tile: 1, lat: 1, lon: 1)>
array([[[12.]]])
Coordinates:
  * lat      (lat) float64 45.3
  * lon      (lon) float64 5.77
  * tile     (tile) int32 1
Attributes:
    units:      m
    long_name:  Soil permeable depth

In [15]:
with xr.set_options(keep_attrs=True):
    ds['SDEP'][0, 0, 0] = SDEP
ds.SDEP

<xarray.DataArray 'SDEP' (tile: 1, lat: 1, lon: 1)>
array([[[12.]]])
Coordinates:
  * lat      (lat) float64 45.3
  * lon      (lon) float64 5.77
  * tile     (tile) int32 1
Attributes:
    units:      m
    long_name:  Soil permeable depth

### Check maximum level before bedrock

In [16]:
ds.DELZ

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.1,  0.2,  0.3,
        0.4,  0.5,  1. ,  3. ,  5. , 15. , 30. ,  5. ])
Coordinates:
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      m
    long_name:  Ground layer thickness

In [17]:
i = 0
while ds.DELZ.cumsum()[i] < SDEP:
    i += 1
    if i > ds.DELZ.shape[0]-1:
        print('The permable depth is greater than the model levels')
        break
        
maxlevel = i # first level of bedrock
maxlevel

17

In [18]:
ds.DELZ.cumsum()

<xarray.DataArray 'DELZ' (layer: 20)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4, 26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20

In [19]:
ds.DELZ.cumsum()[maxlevel]

<xarray.DataArray 'DELZ' ()>
array(26.4)
Coordinates:
    layer    int32 18

In [20]:
ds.DELZ.cumsum()[:maxlevel]

<xarray.DataArray 'DELZ' (layer: 17)>
array([ 0.1,  0.2,  0.3,  0.4,  0.5,  0.6,  0.7,  0.8,  0.9,  1. ,  1.2,
        1.5,  1.9,  2.4,  3.4,  6.4, 11.4])
Coordinates:
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17

In [21]:
ds.DELZ.cumsum()[maxlevel:]

<xarray.DataArray 'DELZ' (layer: 3)>
array([26.4, 56.4, 61.4])
Coordinates:
  * layer    (layer) int32 18 19 20

### Set soil contents and flags
Note: flags are only set in the SAND variable (-3 bedrock / -2 peatland)

In [22]:
# Previous values
ds.SAND[0, :, 0, 0]

<xarray.DataArray 'SAND' (layer: 20)>
array([60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60.,
       60., 60., 60., -3., -3., -3.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      %
    long_name:  Percentage sand content

In [23]:
ds.CLAY[0, :, 0, 0]

<xarray.DataArray 'CLAY' (layer: 20)>
array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.,
       30., 30., 30.,  0.,  0.,  0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      %
    long_name:  Percentage clay content

In [24]:
ds.ORGM[0, :, 0, 0]

<xarray.DataArray 'ORGM' (layer: 20)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      %
    long_name:  Percentage organic matter content

In [25]:
with xr.set_options(keep_attrs=True):
    # Set values until bedrock
    ds.SAND[0, :maxlevel, 0, 0] = SAND
    ds.CLAY[0, :maxlevel, 0, 0] = CLAY
    ds.ORGM[0, :maxlevel, 0, 0] = ORGM
    
    # Set bedrock values
    ds.SAND[0, maxlevel:, 0, 0] = -3 # flag for bedrock
    ds.CLAY[0, maxlevel:, 0, 0] = 0.
    ds.ORGM[0, maxlevel:, 0, 0] = 0.

In [26]:
# Current values
ds.SAND[0, :, 0, 0]

<xarray.DataArray 'SAND' (layer: 20)>
array([60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60., 60.,
       60., 60., 60., -3., -3., -3.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      %
    long_name:  Percentage sand content

In [27]:
ds.CLAY[0, :, 0, 0]

<xarray.DataArray 'CLAY' (layer: 20)>
array([30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30., 30.,
       30., 30., 30.,  0.,  0.,  0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      %
    long_name:  Percentage clay content

In [28]:
ds.ORGM[0, :, 0, 0]

<xarray.DataArray 'ORGM' (layer: 20)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])
Coordinates:
    lat      float64 45.3
    lon      float64 5.77
    tile     int32 1
  * layer    (layer) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20
Attributes:
    units:      %
    long_name:  Percentage organic matter content

In [129]:
ds

<xarray.Dataset>
Dimensions:            (ic: 5, icc: 12, iccp1: 13, iccp2: 14, icp1: 6, lat: 1, layer: 20, lon: 1, slope: 8, tile: 1)
Coordinates:
  * lat                (lat) float64 45.3
  * lon                (lon) float64 5.77
  * tile               (tile) int32 1
  * icp1               (icp1) int32 1 2 3 4 5 6
  * layer              (layer) int32 1 2 3 4 5 6 7 8 ... 13 14 15 16 17 18 19 20
  * ic                 (ic) int32 1 2 3 4 5
  * icc                (icc) int32 1 2 3 4 5 6 7 8 9 10 11 12
  * iccp1              (iccp1) int32 1 2 3 4 5 6 7 8 9 10 11 12 13
  * iccp2              (iccp2) int32 1 2 3 4 5 6 7 8 9 10 11 12 13 14
  * slope              (slope) int32 1 2 3 4 5 6 7 8
Data variables:
    ALBS               (tile, lat, lon) float64 0.5643
    ALIC               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0
    ALVC               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0
    CLAY               (tile, layer, lat, lon) float64 30.0 30.0 ... 0.0 0.0
    CMAS               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    Cmossmas           (tile, lat, lon) float64 0.0
    DELZ               (layer) float64 0.1 0.1 0.1 0.1 0.1 ... 5.0 15.0 30.0 5.0
    DRN                (tile, lat, lon) float64 0.1
    FARE               (tile, lat, lon) float64 1.0
    FCAN               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 1.0 0.0 0.0
    GRO                (tile, lat, lon) float64 0.4285
    LNZ0               (tile, icp1, lat, lon) float64 0.0 0.0 0.0 0.0 0.0 0.0
    ORGM               (tile, layer, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    PAMN               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    PAMX               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    RCAN               (tile, lat, lon) float64 0.0
    RHOS               (tile, lat, lon) float64 0.0
    ROOT               (tile, ic, lat, lon) float64 0.0 0.0 0.0 0.0 0.0
    SAND               (tile, layer, lat, lon) float64 60.0 60.0 ... -3.0 -3.0
    SCAN               (tile, lat, lon) float64 0.0
    SDEP               (tile, lat, lon) float64 12.0
    SNO                (tile, lat, lon) float64 0.0
    SOCI               (tile, lat, lon) float64 15.0
    TBAR               (tile, layer, lat, lon) float64 22.7 26.21 ... 12.61
    TCAN               (tile, lat, lon) float64 11.54
    THIC               (tile, layer, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    THLQ               (tile, layer, lat, lon) float64 0.1392 0.1897 ... 0.0 0.0
    TPND               (tile, lat, lon) float64 0.0
    TSNO               (tile, lat, lon) float64 -273.2
    ZPND               (tile, lat, lon) float64 0.0
    bleafmas           (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    dmoss              (tile, lat, lon) float64 0.0
    fcancmx            (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    grclarea           (lat, lon) float64 100.0
    ipeatland          (tile, lat, lon) float64 0.0
    lfstatus           (tile, icc, lat, lon) float64 1.0 4.0 1.0 ... 0.0 0.0 0.0
    litrmass           (tile, layer, iccp2, lat, lon) float64 0.0 0.0 ... 0.0
    litrmsmoss         (tile, lat, lon) float64 0.0
    nbleafmas          (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ngleafmas_ns       (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    ngleafmas_s        (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nh4_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
    nlitrmass          (tile, iccp1, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    soilnmas           (tile, iccp1, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0
    nmtest             (lat, lon) float64 1.0
    no3_mass           (tile, iccp1, lat, lon) float32 0.0 0.0 0.0 ... 0.0 0.0
    soilpH             (tile, lat, lon) float32 5.835
    nrootmass_ns       (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    nrootmass_s        (tile, icc, lat, lon) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

## Save to netCDF

In [29]:
ds.to_netcdf(path_out+'/FR-Cdp_init_run_bg.nc')
ds.to_netcdf(path_out+'/rsfile_run_bg.nc')